In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:\\Users\\Lenovo\\Documents\\aML\\chatbot'):
   
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\Lenovo\Documents\aML\chatbot\4000_checkpoint.tar
C:\Users\Lenovo\Documents\aML\chatbot\chat-bot.ipynb
C:\Users\Lenovo\Documents\aML\chatbot\cornell_movie_dialogs_corpus.zip
C:\Users\Lenovo\Documents\aML\chatbot\movie_conversations.txt
C:\Users\Lenovo\Documents\aML\chatbot\movie_lines.txt
C:\Users\Lenovo\Documents\aML\chatbot\.ipynb_checkpoints\chat-bot-checkpoint.ipynb
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\.DS_Store
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\chameleons.pdf
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\formatted_movie_lines.txt
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\movie_characters_metadata.txt
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\movie_conversations.txt
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\movie_lines.txt
C:\Users\Lenovo\Documents\aML\chatbot\cornell movie-dialogs corpus\movie_titles_metadata.txt
C:\

> # **IMPORTING LIBARARIES**

In [14]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math


 > # **Load & Preprocess Data**

In [19]:
dir_name="C:\\Users\\Lenovo\\Documents\\aML\\chatbot"
lines_filepath = os.path.join(dir_name,'movie_lines.txt')
conv_filepath = os.path.join(dir_name,'movie_conversations.txt')

In [20]:
#visualize some movie lines...
corpus_name = "cornell movie-dialogs corpus"
with open(lines_filepath,'r',encoding='iso-8859-1') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [21]:
# split each line of the file into Dictionary of fields(lineID,characterID,movieID,character,text)
line_fields = ["lineID","characterID","movieID","character","text"]
lines = {}
with open(lines_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(' +++$+++ ')
        #Extract Fields
        lineObj = {}
        for i,field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj    
            

In [22]:
lines['L1042'],lines['L1043']

({'lineID': 'L1042',
  'characterID': 'u5',
  'movieID': 'm0',
  'character': 'KAT',
  'text': 'Oh, Bianca?  Can you get me my freshman yearbook?\n'},
 {'lineID': 'L1043',
  'characterID': 'u9',
  'movieID': 'm0',
  'character': 'PATRICK',
  'text': "Don ' t you even dare. . .\n"})

In [23]:
# groups fields of lines from 'Loadlines' into conversations based on "movie_conversation.txt"
conv_fields = ["character1ID","character2ID","movieID","utteranceIDs"]
conversations = []
with open(conv_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        #Extract fields
        convObj = {}
       
        for i,field in enumerate(conv_fields):
            convObj[field] = values[i]
        #convert string results from split to list. convObj['utteranceIDs'] == ['L1','L2',...]
            
        lineIds = eval(convObj["utteranceIDs"])
        #reassemble lines
        convObj['lines'] = []
        for lineId in lineIds:
            convObj['lines'].append(lines[lineId]) 
        conversations.append(convObj)

In [24]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': [{'lineID': 'L194',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'lineID': 'L195',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'lineID': 'L196',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'lineID': 'L197',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]}

In [25]:
#extract pair of sentences from conversations
qa_pairs = []
for conversation in conversations:
    for i in range(len(conversation['lines'])-1):
        imputline = conversation['lines'][i]['text'].strip()
        targetline = conversation['lines'][i+1]['text'].strip()
    
        if imputline and targetline:
            qa_pairs.append([imputline,targetline])

In [26]:
len(qa_pairs)

221282

In [27]:
qa_pairs[2]

['Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]

In [28]:
#define path to new file
outdirname= dir_name
datafile = os.path.join(outdirname,"formatted_movie_lines.txt")
delimiter = '\t'
delimiter = str(codecs.decode(delimiter,'unicode_escape'))

#write new csv file
print("\nwriting newly formatted file...")
with open(datafile,'w',encoding = 'utf-8') as outputfile:
    writer = csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print("Done writing to file...")


writing newly formatted file...
Done writing to file...


In [29]:
#visualize some lines
datafile = os.path.join(outdirname,"formatted_movie_lines.txt")
with open(datafile,'rb') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\r\n"
b'Why?\tU

In [30]:
# making the vocabulary
PAD_token = 0  #for padding short senetnces
SOS_token = 1  #start of sentence token<START>
EOS_token = 2  #end of sentence token<END>

class vocabulary:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: 'PAD',SOS_token:'SOS',EOS_token:'EOS'}
        self.num_words = 3 # count PAD,SOS,EOS
        
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words +=1
        else:
            self.word2count[word] += 1
    
    #remove words below a threshhold
    def trim(self, min_count):
        keep_words = []
        for k,v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)
            
        
            

In [31]:
#turn a unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')

In [32]:
#test the function
unicodeToAscii("á Á é É í Í ó")

'a A e E i I o'

In [33]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #Replace any .!? by a whitespace + that character ('!' by ' !'). \1 means the first bracketed group[.!?].
    # r is to not consider \1 as escape character.
    s = re.sub(r"([.!?])", r" \1", s)
    # Remove any character that is nota sequence of lower and upper case letters. + means one or more.
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    #remove sequence of whitespace characters.
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [34]:
# test the function
normalizeString("aaa123aa!s's . dd?")

'aaa aa !s s . dd ?'

In [35]:
datafile = os.path.join(outdirname, "formatted_movie_lines.txt")
# readthe file and split into lines
print("Readaing and processing file... please wait.")
lines = open(datafile, encoding = 'utf-8').read().strip().split('\n')
#split every line into pairs and normalize.
pairs =[[normalizeString(s) for s in pair.split("\t")] for pair in lines]
print("Done reading...")
voc = vocabulary("cornell movie-dialogs corpus")

Readaing and processing file... please wait.
Done reading...


In [36]:
pairs[0]

['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
 'well i thought we d start with pronunciation if that s okay with you .']

In [37]:
# Returns True if both the sentences in a pair 'p' are under the MAX_LENGTH Threshhold.
MAX_LENGTH = 10
def filterPair(p):
    #input sequences must preserve the last word for EOS token
    return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH
#Filter pairs using filter pair conditions.
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [38]:
pairs =[pair for pair in pairs if len(pair)>1]
print("There are {} pairs/conversations".format(len(pairs)))
pairs = filterPairs(pairs)
print("There are {} pairs/conversations".format(len(pairs)))

There are 221282 pairs/conversations
There are 64271 pairs/conversations


In [39]:
#loop through each pair and add question and reply sequence to vocabulary
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Number of words:",voc.num_words)
for pair in pairs[:10]:
    print(pair)
    

Number of words: 18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [40]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs




# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


In [41]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

# It zips each entries with corresponding entries in other list and pads 0 where there is no entry,
# in this process the original matrix is transposed automatically. 
def zeroPadding(l, fillvalue=PAD_token): #pads the matrix with fillvalue(which is 0 here) where it is short, to make it m x n.
    return list(itertools.zip_longest(*l, fillvalue=fillvalue)) #where m is length of longest question which is 10 here.

def binaryMatrix(l, value=PAD_token): # creates binary matrix, if there is a value, 1 is stored else 0 for no value.
    m = []
    for i, seq in enumerate(l):
        m.append([])                #for every list of values there is a list 
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
#1 sorting batch with key as length means the qa_pair with longer question length is before all other pair in the list.
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True) #1
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable) # input transposed when matrix was padded above.
print("lengths:", lengths) # in desc order bcoz of sorting with key as length of question
print("target_variable:", target_variable) # answer corresponds to each question(column) in the input matrix
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  25,    5,    7,   34,  240],
        [  89,  148,   14,   61,   56],
        [  44, 3232,  459,  331,  945],
        [1633,  746,   56,  117,    6],
        [  75,    7,  174,    4,    2],
        [  45,    6,    4,    2,    0],
        [   4,    2,    2,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([8, 7, 7, 6, 5])
target_variable: tensor([[ 334,  101,   77,  100,   56],
        [5879,  315,  271,   80, 4087],
        [ 289,    9,  117,    7,    4],
        [  75, 1160,    6,   59,    2],
        [  83,   77,    2,  170,    0],
        [   6,  969,    0,   40,    0],
        [   2,    4,    0,   47,    0],
        [   0,    2,    0,   36,    0],
        [   0,    0,    0,    6,    0],
        [   0,    0,    0,    2,    0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  Tr

> ## Seq2Seq Model

>  ### Encoder 

Inputs:

    input_seq: batch of input sentences; shape=(max_length, batch_size)...
   
    input_lengths: list of sentence lengths corresponding to each sentence in the batch; shape=(batch_size)...
   
    hidden: hidden state; shape=(n_layers x num_directions, batch_size, hidden_size)

Outputs:

    outputs: output features from the last hidden layer of the GRU (sum of bidirectional outputs); shape=(max_length, batch_size, hidden_size)...
   
    hidden: updated hidden state from GRU; shape=(n_layers x num_directions, batch_size, hidden_size)


In [42]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

> ### Decoder

> attention model

In [43]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        #Element wise multiply the current decoder state with the encoder output and sum them along dim=2.
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # hidden of shape: (1, batch_size, hidden_size)
        #encoder_outputs of shape: (max_length, batch_size, hidden_size)
        
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs) # (max_length,batch_size),after summing along hiddden size(dim=2)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()                           # (batch_size, max_length)

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)    # (batch_size, 1, max_length)

> Luong Attention Decoder

Inputs:

    input_step: one time step (one word) of input sequence batch; shape=(1, batch_size)
    last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)
    encoder_outputs: encoder model’s output; shape=(max_length, batch_size, hidden_size)

Outputs:

    output: softmax normalized tensor giving probabilities of each word being the correct next word in the decoded sequence; shape=(batch_size, voc.num_words)
    hidden: final hidden state of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)


In [44]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        #input_step: one time step (1 word) of input sequence batch, shape = (1,batch_size)
        #last_hidden: final hidden state of encoder's GRU; shape = (n_layers x num_direcions, batch_size, hidden_size)
        #encoder_outputs: encoder model's output; shape = (max_length, batch_size, hidden_size)
        
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        #rnn_output of shape: (1,batch,num_directions x hidden_size)
        #last_hidden of shape: (n_layers x num_direcions, batch_size, hidden_size)
        
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        #(batch_size, 1, max_length) bmm with (batch_size, max_length, hidden) = (batch_size, 1, hidden)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

> ## Traning the model

> Masked LOSS

In [45]:
#This loss function calculates the average negative log likelihood of the elements
#that correspond to a 1 in the mask tensor.

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    return loss, nTotal.item()

In [46]:
#visualisation of loss function
#decoder_output shape:(batcH_size,vocab_size); target size = (batch_size,1)
dec_o = torch.rand(5,7)
print(dec_o)
dec_o = F.softmax(dec_o,dim = 1)
tar = torch.tensor([2,1,5,4,0],dtype = torch.long)
tar = tar.view(-1,1)
mask = torch.tensor([1,0,1,1,0],dtype= torch.uint8)
print(dec_o)
print(tar)
gath_ten = torch.gather(dec_o,1,tar)
print(gath_ten)
print(gath_ten.shape)
crossEntropy = -torch.log(gath_ten)
print("Cross Entropy:", crossEntropy)
mask = mask.unsqueeze(1)
loss = crossEntropy.masked_select(mask)
print("Loss :",loss)
print(loss.shape)
print("sum of the mask element: ",mask.sum())
print("Mean of loss : ", loss.mean())
print("Mean of cross entropy without mask...: ",crossEntropy.mean())

tensor([[0.6065, 0.0988, 0.5122, 0.9869, 0.9087, 0.6228, 0.1591],
        [0.3654, 0.1991, 0.3438, 0.6460, 0.7563, 0.5935, 0.4577],
        [0.9402, 0.7217, 0.0872, 0.0887, 0.9356, 0.6957, 0.4502],
        [0.2010, 0.4043, 0.2351, 0.3570, 0.0830, 0.2276, 0.0629],
        [0.2761, 0.3901, 0.7730, 0.1016, 0.5399, 0.9729, 0.5490]])
tensor([[0.1432, 0.0862, 0.1303, 0.2095, 0.1937, 0.1455, 0.0915],
        [0.1253, 0.1061, 0.1226, 0.1659, 0.1852, 0.1574, 0.1374],
        [0.1981, 0.1592, 0.0844, 0.0845, 0.1972, 0.1551, 0.1214],
        [0.1386, 0.1698, 0.1434, 0.1620, 0.1232, 0.1423, 0.1207],
        [0.1084, 0.1215, 0.1781, 0.0910, 0.1411, 0.2175, 0.1424]])
tensor([[2],
        [1],
        [5],
        [4],
        [0]])
tensor([[0.1303],
        [0.1061],
        [0.1551],
        [0.1232],
        [0.1084]])
torch.Size([5, 1])
Cross Entropy: tensor([[2.0378],
        [2.2433],
        [1.8635],
        [2.0943],
        [2.2222]])
Loss : tensor([2.0378, 1.8635, 2.0943])
torch.Size([3])


..\aten\src\ATen\native\LegacyDefinitions.cpp:29: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


sum of the mask element:  tensor(3)
Mean of loss :  tensor(1.9985)
Mean of cross entropy without mask...:  tensor(2.0922)


In [47]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    #input_variable = input_variable.to(device)
    #lengths = lengths.to(device)
    #target_variable = target_variable.to(device)
    #mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    #decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            #decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [48]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(outdirname, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [49]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], dtype=torch.long)
        all_scores = torch.zeros([0])
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [50]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    #input_batch = input_batch.to(device)
    #lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

**Run Model**

In [51]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(outdirname, model_name, corpus_name,
                          # '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                           # '{}_checkpoint.tar'.format(4000))

loadFilename = os.path.join(dir_name,'4000_checkpoint.tar')
# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
#encoder = encoder.to(device)
#decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


**Run Training**

In [52]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
#for state in encoder_optimizer.state.values():
    #for k, v in state.items():
        #if isinstance(v, torch.Tensor):
            #state[k] = v.cuda()

#for state in decoder_optimizer.state.values():
    #for k, v in state.items():
        #if isinstance(v, torch.Tensor):
            #state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, outdirname, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


*Run Evaluation*

In [53]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> hey
Bot: what ? ? ? ? ?
> what is your name?
Bot: victor frankenstein . is . . .
> fuck
Bot: i m sorry . is .
> where are you?
Bot: i m leaving . s in . .
> shutup
Error: Encountered unknown word.
> shut it
Bot: it s okay . is it ?
> shut up
Bot: i m sorry . is .
> okay go now
Bot: okay . is all right . .
> q


In [54]:
print("END")

END
